# Tensor network emulators

First, create an environment where the jupyter notebook can run. The next cell does that for you.

In general, the places where you should insert your code are marked by the dots `...`.

This does not mean that you can complete the task with a one-liner, you might require more lines.
The dots are also used because some specific variable names are required to be able to perform the final check.

In [ ]:
!python3 -m venv ./tn_env
!source tn_env/bin/activate
!pip install numpy, matplotlib
!git clone https://baltig.infn.it/quantum_tea_leaves/py_api_quantum_tea_leaves.git
!git clone https://baltig.infn.it/quantum_matcha_tea/py_api_quantum_matcha_tea.git
!pip install -e ./py_api_quantum_tea_leaves
!pip install -e ./py_api_quantum_matcha_tea

In [2]:
import numpy as np

yes =  "\033[1;32m" + u"\N{check mark}" #+ "\033[1;30m"
no =  "\033[1;31m" + u"\N{ballot x}" #+ "\033[1;30m"

## Basic handling of tensors

### Handling different shapes

<details>
  <summary>Solution</summary>

```python
def my_reshape(vec, shape, idxs):
    return vec[ idxs[0] + shape[0]*idxs[1] ]
```
</details>

In [3]:
vector = np.arange(16, dtype=int)
shape = (4, 4)
idxs = (2, 2)
id

def my_reshape(vec, shape, idxs):
    """
    Return the element of the vector `vec` at position `idxs`
    as if the vector is a matrix of shape `shape`

    Parameters
    ----------
    vec : np.ndarray
        Vector of shape (n,)
    shape : tuple
        New shape of the vector
    idxs : tuple
        Indexes to access the array value in the
        reshaped position

    Returns
    -------
    int
        value of the array at indexes `idxs`
    """
    return vec[ idxs[0] + shape[0]*idxs[1]]

if vector.reshape(shape)[idxs] == my_reshape(vector, shape, idxs):
    print( yes + " Succed!")
else:
    print(  no + f" Element {vector.reshape(shape)[idxs]} is not the same of your element {my_reshape(vector, shape, idxs)}")

✓ Succed!


### Permutation

Notice that numpy calls `np.transpose` what we call permutation, since it is the multi-dimensional generalization of the matrix transposition.

If you have different indexes $x_1, \dots, x_n$ defined in the ranges $x_1 \in [0, ..., b_1-1], ..., x_n \in [0, ..., b_n-1]$
then you can write the integer $i$ index that runs over the whole tensor as:
$$
i = x_n + b_n x_{n-1} + b_n b_{n-1} x_{n-2} + ... + b_n b_{n-1}...b_2 x_1
$$

<details>
  <summary>Hint part 1</summary>
  
  The ranges are the shapes.

  The index at the position 0 is equivalent to $x_n$ of the formula
  reported in the comments. The function `np.cumprod` might be useful.
</details>

<details>
  <summary>Solution part 1</summary>

```python
def get_int_idx(shape, idxs):
    cumulative_shape = np.cumprod(shape)
    idxs = np.array(idxs)
    int_idx = idxs[0] + (idxs[1:]*cumulative_shape[:-1]).sum()
    return int_idx
```
</details>

<details>
  <summary>Hint part 2</summary>

```python
def my_permute(tensor, permutations):
    shape = np.array(tensor.shape)
    permutations = np.array(permutations)
    # Initialize a tensor with the correct shape, which is the permuted shape of the original tensor
    new_tensor = np.zeros( ... )
    # The permutation is a copy with a different order
    for ii in range(shape[0]):
        for jj in range(shape[1]):
            for kk in range(shape[2]):
                new_tensor[...] = tensor[...]

    return new_tensor
```
</details>

<details>
  <summary>Solution part 2</summary>

```python
def my_permute(tensor, permutations):
    shape = np.array(tensor.shape)
    permutations = np.array(permutations)
    # Initialize a tensor with the correct shape, which is the permuted shape of the original tensor
    new_tensor = np.zeros(shape[permutations] )
    for ii in range(shape[0]):
        for jj in range(shape[1]):
            for kk in range(shape[2]):
                new_tensor[ii, kk, jj] = tensor[ii, jj, kk]

    return new_tensor
```
</details>

In [31]:
# The formula is as follows: if you have x_1, ..., x_n
# in the ranges x_1 ∈ {0, ..., b_1-1}, ..., x_n ∈ {0, ..., b_n-1}
# then the integer i is defined as:
# i = x_n + b_n x_{n-1} + b_n b_{n-1} x_{n-2} + ... + b_n b_{n-1}...b_2 x_1

shape = (2, 3, 4)
idxs = (1, 0, 0)
vector = np.arange(np.prod(shape))

def get_int_idx(shape, idxs):
    """
    Return the integer index corresponding to the tensor index `idxs`
        for a tensor of shape `shape`

    Parameters
    ----------
    shape : tuple
        Shape of the tensor
    idxs : tuple
        Tensor indexes

    Returns
    -------
    int
        Integer index corresponding to the tensor index `idxs`
        for a tensor of shape `shape`
    """
    shape = np.array(shape)
    idxs = np.array(idxs)
    cumulative_sum = np.cumprod(shape[::-1])
    rev_idx = idxs[::-1]
    print(rev_idx)
    print(cumulative_sum[:-1])
    int_idx= rev_idx[0] + (rev_idx[1:]*cumulative_sum[:-1]).sum()
    return int_idx

if vector.reshape(shape)[idxs] == vector[get_int_idx(shape, idxs)]:
    print( yes + " Succed!")
else:
    print(  no + f" Element {vector.reshape(shape)[idxs]} is not the same of your element {vector[get_int_idx(shape, idxs)]}")


permutations = (0, 2, 1)
tensor = vector.reshape(shape)

def my_permute(tensor, permutations):
    """
    Permute the indexes of the tensor `tensor`
    using the permutation `permutations`

    Parameters
    ----------
    tensor : np.ndarray
        Rank-3 tensor
    permutations : tuple
        Permutation of the tensor indexes

    Returns
    -------
    np.ndarray
        Permuted tensor
    """
    shape = np.array(tensor.shape)
    permutations = np.array(permutations)
    new_tensor = np.zeros(shape[permutations])
    for ii in range(shape[0]):
        for jj in range(shape[1]):
            for kk in range(shape[2]):
                new_tensor[ii,kk,jj] =tensor[ii, jj, kk] 

    return new_tensor

if np.isclose( tensor.transpose(permutations), my_permute(tensor, permutations) ).all() :
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

[0 0 1]
[ 4 12]
✓ Succed!
✓ Succed!


### Contractions

Perform the tensor contraction of the following tensors using matrix-matrix multiplication (you should not use `np.tensordot` at this level.):

1. The Index `1` of tensor `tens_1` is contracted with the indexes `(0, 1)` of tensor `tens_2`
   <details>
   <summary>Hint part 1</summary>
   You should merge the indexes of `tens_2` using a reshape
   </details>
2. The Index `1` of tensor `tens_1` is contracted with the indexes `0` of tensor `tens_2`
   <details>
   <summary>Hint part 2</summary>
   You should merge the indexes of `tens_2` using a reshape
   </details>
3. The Index `1` of tensor `tens_1` is contracted with the indexes `2` of tensor `tens_2`
   <details>
   <summary>Hint part 3</summary>
   On top of what you did before, you have to transpose something on `tens_2`
   </details>
4. The Index `(0, 1)` of tensor `tens_1` is contracted with the indexes `(1, 2)` of tensor `tens_2`
   <details>
   <summary>Hint part 4</summary>
   Transpose `tens_1` and `tens_2`, reshape them and you are there!
   </details>

Recall that in python the symbol `@` is a matrix-matrix multiplication. Thus, `np.matmul(A, B) = A @ B`.



In [74]:
def get_tens(shape):
    return np.arange(np.prod(shape)).reshape(shape)


# ----------------------------- 1 -----------------------------
shape_1, shape_2 = (4, 6), (2, 3, 8)
tens_1, tens_2 = get_tens(shape_1), get_tens(shape_2)

result = tens_1@ np.reshape(tens_2, [2*3, 8])

if np.isclose( np.tensordot(tens_1, tens_2.reshape(6, 8), ([1], [0])), result ).all() :
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

# ----------------------------- 2 -----------------------------
shape_1, shape_2 = (4, 2), (2, 3, 8)
tens_1, tens_2 = get_tens(shape_1), get_tens(shape_2)

result = np.tensordot(tens_1, tens_2, ([1], [0]))

if np.isclose( np.tensordot(tens_1, tens_2, ([1], [0])), result ).all() :
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

# ----------------------------- 3 -----------------------------
shape_1, shape_2 = (4, 6), (2, 3, 6)
tens_1, tens_2 = get_tens(shape_1), get_tens(shape_2)

result = np.tensordot(tens_1, tens_2, ([1], [2]))

if np.isclose( np.tensordot(tens_1, tens_2, ([1], [2])), result ).all() :
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

# ----------------------------- 4 -----------------------------
shape_1, shape_2 = (4, 6, 10), (2, 4, 6)
tens_1, tens_2 = get_tens(shape_1), get_tens(shape_2)

result = np.tensordot(tens_1, tens_2 , ([0,1], [1,2]))

if np.isclose( np.tensordot(tens_1, tens_2, ([0, 1], [1, 2])), result ).all() :
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

✓ Succed!
✓ Succed!
✓ Succed!
✓ Succed!


### Singular value decomposition

Starting from the order 4 tensor `tens` apply an SVD splitting legs (0, 1) and (2, 3).

1. Check that the singular values are `[s_1, s_2, 0, 0]`
2. Contract the singular values to the `V_mat` matrix (right matrix);
3. reshape `U_mat` into a (1, 2, n) tensor and `V_mat` in a (n, 2, 1) tensor;
4. Contract the tensor network by first contracting `U_tens` and then `V_tens`;
5. Can you obtain the same result using only `V_tens`? And only `U_tens`? Why?

In [9]:
tens = np.zeros(2**4)
tens[[0, -1]] = 1/np.sqrt(2)
tens = tens.reshape([2]*4)
print(tens)
tens = tens.reshape([4,4])
print(tens)

tens = np.reshape(tens, [4,4])
print(tens)

U_mat, singvals, V_mat = np.linalg.svd(tens)
V_mat = np.tensordot(singvals,V_mat, ([0],[0]))
np.shape(U_mat)
U_tens = U_mat.reshape((1,2,-1))
V_tens = U_mat.reshape((-1,2,1))

print(U_tens)
print(V_tens)

result_contraction = 0

if np.isclose( result_contraction, 1 ):
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

[[[[0.70710678 0.        ]
   [0.         0.        ]]

  [[0.         0.        ]
   [0.         0.        ]]]


 [[[0.         0.        ]
   [0.         0.        ]]

  [[0.         0.        ]
   [0.         0.70710678]]]]
[[0.70710678 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.70710678]]
[[0.70710678 0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.70710678]]
[[[1. 0. 0. 0. 0. 0. 1. 0.]
  [0. 0. 0. 1. 0. 1. 0. 0.]]]
[[[1.]
  [0.]]

 [[0.]
  [0.]]

 [[0.]
  [0.]]

 [[1.]
  [0.]]

 [[0.]
  [0.]]

 [[0.]
  [1.]]

 [[0.]
  [1.]]

 [[0.]
  [0.]]]
✗ Failed! 


### Image compression with SVD

Apply the SVD to the image, and before contracting the singular value to `V_mat` cut a part of them.

How many singvals can you cut while still enjoying the Shiba blinking?

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

im_frame = Image.open('shibapng.png').convert('1')
image = np.array(im_frame.getdata())
x = int(np.sqrt(len(image)))
image = image.reshape(-1, 820)

plt.imshow(image)
plt.show()

In [ ]:
U_mat, singvals, V_mat = np.linalg.svd(image)

U_mat = ...
singvals = ...
V_mat = np.tensordot(...)

image = ...

plt.imshow(image)
plt.show()

## Quantum emulator

### Write your own MPS

Write the MPS state of the following quantum state:

$$
|\psi\rangle = \frac{1}{\sqrt 2}\left(|000\rangle + |111\rangle\right)
$$

1. Apply the SVD to the tensor of leg (0) w.r.t. legs 1,2
2. Write `U_tens` as a (1, 2, n) tensor
3. Apply the SVD to `V_mat` of legs (0, 1) w.r.t. 2

In [ ]:
sigma_z = np.array([[1, 0], [0, -1]])
sigma_x = np.array([[0, 1], [1, 0]])
cx = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

In [ ]:
from qtealeaves.emulator import MPS
from qtealeaves.convergence_parameters import TNConvergenceParameters

num_qubs = 3
tens = np.zeros(2**num_qubs)
tens[[0, -1]] = 1/np.sqrt(2)
mps_list = []


# Divide first qubit from the rest of the state
U_mat, singvals, V_mat = np.linalg.svd(...)
V_mat = np.tensordot(...)

U_tens = ...
V_tens = ...
mps_list.append(U_tens)

# Divide first qubit from the rest of the state
U_mat, singvals, V_mat = np.linalg.svd(V_tens, ...)
V_mat = np.tensordot(...)

U_tens = ...
mps_list.append(U_tens)

V_tens = ...
mps_list.append(V_tens)


mps_state = MPS.from_statevector(tens.reshape(-1))

if [np.isclose( tens1, tens2 ).all() for tens1, tens2 in zip(mps_list, mps_state)].all():
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

### Application of gates to the MPS

Starting from the state $|00...0\rangle$ go to the GHZ state applying single and two-qubits gates.
Use the native function to shift the isometry center.

In [ ]:
num_qubits = 10
conv_params = TNConvergenceParameters(max_bond_dimension=128)
mps_state = MPS(num_qubits, conv_params)

ghz = np.zeros(2**num_qubs)
ghz[[0, -1]] = 1/np.sqrt(2)


site_0 = mps_state[0]
# Apply the single qubit gate
site_0_new = ...

mps_state._tensors[0] = site_0_new

# Apply the two-qubit gates
for ii in range(num_qubits-1):
    # Move the isometry center. It is actually moved automatically
    # due to how we apply the two-qubit gates, but it would be necessary
    # if the gates were not always between i,i+1
    mps_state.site_canonize(ii, True)

    site_ii = mps_state[ii]
    site_ii_1 = mps_state[ii+1]

    # Contract the two qubits
    two_qubits = ...
    # Contract the resulting tensor with the gate
    tensors_after_application = ...
    # Split the gate
    U_mat, singvals, V_mat = np.linalg.svd(...)

    # Truncate the singular values, keep only the singvals > 0
    U_mat = ...
    singvals = ...
    V_mat = np.tensordot(...)

    # Reshape in the correct order
    U_tens = ...
    V_tens = ...

    mps_state._tensors[ii] = U_tens
    mps_state._tensors[ii+1] = V_tens
    mps_state._singvals[ii+1] = singvals

if np.isclose(np.vdot(mps_state.to_statevector(), ghz), 1):
    print( yes + " Succed!")
else:
    print(  no + f" Failed! ")

# Using Quantum Matcha TEA

In [ ]:
# Import necessary modules
from qiskit import QuantumCircuit
import qtealeaves.observables as obs
from qmatchatea import QCConvergenceParameters, run_simulation, QCBackend
from qmatchatea.qk_utils import QFT_qiskit

Prepare the quantum circuit you want to emulate

In [ ]:
num_qubits = 20

qc = QuantumCircuit(num_qubits)

state = "product"

if state == "product":
    # Add your code here to prepare your choice quantum state. For example apply a QFT to product state
    for ii in range(num_qubits):
        qc.h(ii)
elif state == "ghz":
    # Add your code here to prepare your choice quantum state. For example apply a QFT to ghz state
    qc.h(0)
    for ii in range(num_qubits-1):
        qc.cx(ii, ii+1)
elif state == "random":
    # Add your code here for the random state. Is it as fast as the other two?
    ...
    # try other initialization if you want


# Appy a QFT
QFT_qiskit(qc, num_qubits)

print("Circuit defined!")

It is possible to choose between different approaches for running the simulation:

- The backend can be either python "PY" or fortran "FR".
- The machine precision can be either double complex "Z" or single complex "C".
- The device of the simulation can be either "cpu" or "gpu".

In [ ]:
backend = QCBackend(
    backend="PY", precision="Z", device="cpu", num_procs=1, mpi_approach="SR"
)

Define the convergence parameters, in particular the maximum bond dimension.
The maximum bond dimension controls how much entanglement we can encode in
the system while still obtaining trustful results.
The maximum bond dimension reachable for a given number of qubits :math:`n`
is $\chi=2^{\lfloor\frac{n}{2}\rfloor}$. As you see, if we want to
encode *any* state we still have an exponential scaling.


In [ ]:
conv_params = QCConvergenceParameters(max_bond_dimension=1024, singval_mode="C")

It is easy to define the observables for a quantum matcha tea simulation using the observables available in qtealeaves.
You can finde all the available at the [qtealeaves observables documentation](https://quantum_tea_leaves.baltig-pages.infn.it/py_api_quantum_tea_leaves/chapters/measurements.html).

In [ ]:
# Container of all the observables
observables = obs.TNObservables()

# Local observables
observables += obs.TNObsLocal("sigma_z", "Z")

# Entanglement entropy
observables += obs.TNObsBondEntropy()

# Projective measurements
observables += obs.TNObsProjective(1024)

# Probability measurement
observables += obs.TNObsProbabilities()

# Tensor product observable, such as the parity of the whole state
observables += obs.TNObsTensorProduct("parity", ["Z" for _ in range(num_qubits)], [[ii] for ii in range(num_qubits)])

Run the simulation and obtain the results

In [ ]:
results = run_simulation(
    qc,
    convergence_parameters = conv_params,
    observables=observables,
    backend=backend
    )

Inspect the results to extract the informations you are interested about

In [ ]:
for key, val in results.observables.items():
    print("-"*50, key, "-"*50)
    print(val)
    print()

Check if there was any truncation during the simulation

In [ ]:
print(f"The fidelity of the simulation is {results.fidelity}")